# Latent Dirichlet Allocation with Scikit-Learn

In this notebook I'll use Scikit-Learn to perform Latent Dirichlet Allocation on the 20 Newsgroups dataset. The implementation in Scikit-Learn uses the online variational Bayes algorithm. I'll also explain the Gibbs sampling approach and present an example that illustrates intuitively why it works. 

In [177]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import stop_words, text
from pprint import pprint
import numpy as np

The fetch_20newsgroups function will by default shuffle the data and import just the training data. 

In [126]:
newsgroups_data = fetch_20newsgroups(random_state = 1, remove = ('headers', 'footers', 'quotes'))

In [130]:
pprint(list(newsgroups_data.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


Let's have a look at some of the documents:

In [285]:
newsgroups_data.data[:2]

["Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n",
 "\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap

In [149]:
newsgroups_data.target[:10]

array([17,  0, 17, 11, 10, 15,  4, 17, 13, 12])

In [152]:
unique, counts = np.unique(newsgroups_data.target, return_counts = True)
dict(zip(unique, counts))

{0: 480,
 1: 584,
 2: 591,
 3: 590,
 4: 578,
 5: 593,
 6: 585,
 7: 594,
 8: 598,
 9: 597,
 10: 600,
 11: 595,
 12: 591,
 13: 594,
 14: 593,
 15: 599,
 16: 546,
 17: 564,
 18: 465,
 19: 377}

The object CountVectorizer combined with the method fit_transform will take in our documents and output a matrix of size (number of documents) x (vocabulary size) where the $i,j$ entry is the number of times word $j$ appears in document $i$. The size of the vocabulary will depend on whether or not we decide to use stop words and what the values of max_df and min_df are. Setting stop_words = "english" as an input to the CountVectorizer object will tell it to ignore a certain fixed list of common English words. The options max_df and min_df are either integers or floats between 0 and 1.0: if a word is found in greater than max_df documents, we ignore it; similarly, if a word is found in less than min_df documents, we ignore it. By setting appropriate values of max_df and min_df, we allow CountVectorizer to find the common words in our specific corpus. We'll try both options. It's also good to note that CountVectorizer by default sets the token pattern to \b\w\w+\b, from which we can see that it ignores punctuation. 

In [178]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(["like", "does", "going", "said", "say", "did", "use"])

In [249]:
vectorizer = CountVectorizer(stop_words = my_stop_words, max_df =.3, min_df = 3)
tf = vectorizer.fit_transform(newsgroups_data.data)

In [250]:
tf.shape

(11314, 26740)

In Latent Dirichlet Allocation we must provide a number of topics (n_components) for the procedure to learn. We'll choose the online variational Bayes method.

In [251]:
lda = LatentDirichletAllocation(n_components=20, max_iter=10, learning_method='online', random_state=0)

In [252]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [253]:
tf_feature_names = vectorizer.get_feature_names() # gets tokens found by CountVectorizer (the columns of tf)
# we'll print out the 10 top words per topic
for index, topic in enumerate(lda.components_):
    top_words = " ".join([tf_feature_names[i] for i in topic.argsort()[: -11:-1]])
    print("Topic", index, ":" , top_words)

Topic 0 : armenian armenians turkish people went turkey came armenia turks didn
Topic 1 : team year game games play season hockey league players win
Topic 2 : don just know think people time good way ve want
Topic 3 : window output widget entry motif program file server mouse set
Topic 4 : fpu hr uiuc 147 200 processor vt 250 500 hst
Topic 5 : better left points nazi bad run fans men definitely time
Topic 6 : edu file available information mail files com ftp program software
Topic 7 : key used new car good encryption need using chip high
Topic 8 : max edu com 34 9v au mk gov cs nist
Topic 9 : god jesus christian bible church believe christians faith christ life
Topic 10 : space israel war jews nasa israeli world earth launch 000
Topic 11 : bike 000 edu new university health medical 1993 research pain
Topic 12 : pl ms uw tm mr mw mp ww mm q6
Topic 13 : cx w7 mv chz t7 bh lk hz ck c_
Topic 14 : greek water jim weapon frank street greece dept western william
Topic 15 : drive card bit disk

The transform method for lda will return a matrix of size (number of documents) x (n_components) which reflects the document-topic distribution. 

In [254]:
tf_transform = lda.transform(tf)
print(tf_transform.shape)
print(tf_transform[0])

(11314, 20)
[0.00081967 0.00081967 0.25336552 0.00081967 0.00081967 0.00081967
 0.00081967 0.00081967 0.00081967 0.00081967 0.58427815 0.00081967
 0.00081967 0.00081967 0.00081967 0.00081967 0.14842191 0.00081967
 0.00081967 0.00081967]


In [255]:
lda.perplexity(tf) # perhaps not a great metric for evaluating the model

3135.236527202665

The components_ method outputs a matrix of size (n_components) x (number of tokens)

In [256]:
lda.components_.shape

(20, 26740)

In [257]:
max_topic = np.argmax(tf_transform, axis=1) # selects the most likely topic per document

In [258]:
max_topic[:100]

array([10,  2, 10,  7, 11,  2, 12,  2,  2,  7,  6,  7, 11,  5,  2,  2, 16,
        8,  4,  2,  7,  5, 16,  5,  1,  9,  7,  7, 19,  2,  7,  2,  7,  9,
       10, 17,  2, 17,  2,  2, 15, 11,  7,  6, 12,  6,  6,  7,  2, 15,  7,
        0, 17,  7,  7, 10,  7,  2,  7, 16,  7,  2, 14,  2,  7,  7,  7,  5,
       17, 16, 11, 11, 15,  2, 15,  7,  2,  7, 14,  2, 16,  2,  7,  7, 15,
        2,  7, 11,  2,  0, 15, 10, 15,  2,  2,  9,  0,  7, 11,  7])

In [259]:
unique, counts = np.unique(max_topic, return_counts = True)
dict(zip(unique, counts))

{0: 421,
 1: 460,
 2: 3602,
 3: 154,
 4: 30,
 5: 182,
 6: 1176,
 7: 1762,
 8: 61,
 9: 446,
 10: 455,
 11: 349,
 12: 17,
 13: 27,
 14: 43,
 15: 1324,
 16: 638,
 17: 36,
 18: 12,
 19: 119}

The topic categories are very unbalanced - the topic having top words that are very generic (model topic number 2) is being chosen as the most likely topic for a disproportionate number of documents. Let's take original topic number 17 (mideast) and see what topics from above the model attributed to those documents.

In [321]:
def get_max_topics(topic_number):
    indices = [i for i in range(len(newsgroups_data.target)) if newsgroups_data.target[i] == topic_number]
    max_topics = [row.argmax() for row in tf_transform[indices]]
    return max_topics

get_max_topics(17)[:10]

[10, 10, 2, 10, 2, 2, 2, 0, 16, 10]

In [322]:
unique, counts = np.unique(get_max_topics(17), return_counts = True)
dict(zip(unique, counts))

{0: 76,
 1: 4,
 2: 197,
 3: 2,
 5: 2,
 6: 5,
 7: 5,
 9: 9,
 10: 206,
 11: 4,
 12: 1,
 14: 6,
 16: 43,
 17: 1,
 19: 3}

This distribution could make some sense - excluding topic 2 (which is very generic), the other topics that get confused with topic 17 (mideast) have very related words. There are likely many other things you could do to improve the results of this model, but we'll stop here. 

I'll now explain the Gibbs sampling approach for the assignment of words to topics, following the notation from here: https://people.cs.umass.edu/~wallach/courses/s11/cmpsci791ss/readings/griffiths02gibbs.pdf. Suppose we have the following set up:

\begin{align*}
\theta^{(d_i)} &\sim Dir(\alpha) \\
z_{i}|\theta^{(d_i)} &\sim Mult(\theta^{(d_i)})\\
\phi^{(z_{i})} &\sim Dir(\beta) \\
w_{i}|z_{i}, \phi^{(z_{i})} &\sim Mult(\phi^{(z_{i})})
\end{align*}

where the topic for word $i$ in document $d_i$, $z_{i}$, is drawn from a multinomial distribution parametrized by $\theta^{(d_i)}$ and $\theta^{(d_i)}$ has a Dirichlet prior parametrized by $\alpha$; similarly, given a topic $z_{i}$, word $w_{i}$ is drawn from a multinomial distribution parametrized by $\phi^{(z_{i})}$ where $\phi^{(z_{i})}$ has a Dirichlet prior parametrized by $\beta$. Let $\mathbf{w}$ be our vocabulary.

Our input will be a matrix of size (number of documents) x (size of vocabulary) (call this the DW matrix) where the $i, j$ entry tells you the number of times word $j$ appears in document $i$. In Gibbs sampling, we first initialize randomly by assigning each word in each document to a topic. This gives us a word-topic matrix WT where the $i, j$ entry tells you how many times word $i$ was assigned to topic $j$. Finally, we have a document-topic matrix DT where the $i, j$ entry tells you the number of words in topic $j$ in document $i$. We then go through each document and each word and reassign it's topic according to the relative probabilities below. A careful computation will show that:

$$ P(z_i = j|z_{-i}, \mathbf{w}) \propto \frac{n^{(w_i)}_{-i,j} + \beta}{n_{-i,j}^{(\cdot)} + W\beta} \cdot \frac{n^{(d_i)}_{-i,j} + \alpha}{n_{-i, \cdot}^{(d_i)} + T\alpha} $$

where $z_{-i}$ is an assignment of topics for all words except $w_i$, $W$ is the size of the vocabulary, $T$ is the number of topics, $n^{(w_i)}_{-i,j}$ is the total number of times word $w_i$ is assigned to topic $j$, $n_{-i,j}^{(\cdot)}$ is the total number of words assigned to topic $j$, $n^{(d_i)}_{-i,j}$ is the number of words from document $d_i$ assigned to topic $j$, and finally $n_{-i, \cdot}^{(d_i)}$ is the total number of words in document $d_i$ (and we enforce that all of these counts exclude the current word). For a derivation of this formula, see the pdf linked to above. 

Let's get some intuition for why this procedure reaches a stable distribution by considering a very simple example. Suppose you have two documents $d_1$ and $d_2$. Now suppose that $d_1$ is "a a a a a" and $d_2$ is "b b b b b". Suppose we have two topics $t_1$ and $t_2$ and we randomly initialize by labelling the first three words of both documents by $t_1$ and the latter two words of both documents by $t_2$. In this degenerate case we can ignore the Dirichlet priors (indeed, by taking the limit as $\beta, \alpha \rightarrow 0$ we obtain two binomial priors). The above probability will become $ P(z_i = j|z_{-i}, \mathbf{w}) \propto \frac{n^{(w_i)}_{-i,j}}{n_{-i,j}^{(\cdot)}} \cdot \frac{n^{(d_i)}_{-i,j}}{n_{-i, \cdot}^{(d_i)}}$. We would hope that ultimately all the words "a" are assigned to one topic with high probability and likewise for word "b" and the other topic. In order to see the pattern here faster, let's assume that at each update step we select the topic with the higher relative probability.

word 1 document 1: $P(\text{topic} = t_1) \propto \frac{2}{5} \cdot \frac{2}{4} = \frac{1}{5}$ and $P(\text{topic} = t_2) \propto \frac{2}{4} \cdot \frac{2}{4} = \frac{1}{4}$. So, word 1 in document 1 is now assigned to $t_2$. The first factor is the probability of the word given the topic, so it considers how common the topic assignment is in the other documents, while the second term is the probability of the topic given the document, so it considers how important the topic is in the document at hand.

word 2 document 1: $P(\text{topic} = t_1) \propto \frac{1}{4} \cdot \frac{1}{4} = \frac{1}{16}$ and $P(\text{topic} = t_2) \propto \frac{3}{5} \cdot \frac{3}{4} = \frac{9}{20}$. So, word 2 in document 1 is now assigned to $t_2$.

And in this way, we can continue and be convinced that the words will be assigned to topics in a coherent manner. 